# MGT-530 : Group 5 Final Project

- [Scenario 1: Distance Weighted Demand](##Scenario-1-Distance-Weighted-Demand)

In [9]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.colors as mcolors


In [3]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'

In [4]:
vehicle_matrix = pd.read_excel(repo_url + 'vhc_matrix_city_excluded.xlsx')
vehicle_matrix = vehicle_matrix.iloc[0:, :1]
vehicle_matrix

,Places
0,16
1,91
2,50
3,20
4,127
5,100


In [5]:
v = vehicle_matrix.values.tolist()
v = [item[0] for item in v]

In [6]:
full_data = pd.read_csv(repo_url + 'full_data.csv')

## Code to display the map

In [7]:
npa_coords = full_data[['NPA', 'lat', 'lng']]
depot_row = pd.DataFrame([{'NPA': 0, 'lat': 46.60396069250175, 'lng': 6.538034286509177}])
npa_coords = pd.concat([npa_coords, depot_row], ignore_index=True)
npa_to_coords = npa_coords.set_index('NPA')[['lat', 'lng']].to_dict('index')

In [8]:
def get_color_palette(n):
    palette = sns.color_palette("pastel", n)
    return [mcolors.rgb2hex(color) for color in palette]

def plot_routes_folium(df_results, npa_to_coords):
    depot_coords = npa_to_coords[0]
    m = folium.Map(location=[depot_coords['lat'], depot_coords['lng']], zoom_start=11)
    num_vehicles = df_results[df_results['vehicle_id'] != 'Total'].shape[0]
    colors = get_color_palette(num_vehicles)

    for idx, row in df_results.iterrows():
        if row['vehicle_id'] == 'Total':
            continue
        fg = folium.FeatureGroup(name=f"Vehicle {row['vehicle_id']}")
        route = row['route']
        points = []
        for npa in route:
            try:
                npa_int = int(npa)
            except Exception:
                continue
            coords = npa_to_coords.get(npa_int)
            if coords:
                points.append((coords['lat'], coords['lng']))
        if points:
            folium.PolyLine(
                points,
                color=colors[idx % len(colors)],
                weight=5,
                opacity=1,  # Fully opaque
                popup=f"Vehicle ID: {row['vehicle_id']}"
            ).add_to(fg)
            for lat, lng in points:
                folium.CircleMarker([lat, lng], radius=3, color='black').add_to(fg)
        fg.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    return m

## Scenario 1 Distance Weighted Demand

In [29]:
def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Collects solution in a DataFrame for easy CSV export, with NPA in route."""
    results = []
    total_distance = 0
    total_load = 0
    unused_vehicles = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        capacity = data['vehicle_capacities'][vehicle_id]

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            route_load += demand
            route.append(node_to_npa[node_index])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route.append(node_to_npa[end_node])

        if route_load == 0:
            unused_vehicles.append(vehicle_id)
            continue  # Skip empty routes

        results.append({
            "vehicle_id": vehicle_id,
            "capacity": capacity,
            "route": route,
            "distance_km": route_distance ,
            "load": route_load,
        })

        total_distance += route_distance
        total_load += route_load

    # Add summary row
    results.append({
        "vehicle_id": "Total",
        "capacity": "",
        "route": "",
        "distance_km": total_distance,
        "load": total_load,
    })

    df_results = pd.DataFrame(results)
    print(df_results)
    return df_results

## Scenario 2 Full Demand

### Wednesday

In [15]:
demand_wed_df = pd.read_csv(repo_url + 'demand_wed_full.csv', header=None)
# demand_wed = pd.read_csv(repo_url + 'demand_wed.csv',header=None)
demand_wed_df

,0,1,2,3,4,5
0,index,Commune,NPA,Commune d'annonce / District,full_demand_wed,distance_to_venoge_km
1,0,Penthaz,1303,Penthaz,100.0,0.74
2,0,Penthaz,1303,Penthaz,100.0,0.74
3,0,Penthaz,1303,Penthaz,46.0,0.74
4,1,Penthalaz,1305,Penthalaz,100.0,2.72
...,...,...,...,...,...,...
326,232,Chesières,1885,Ollon,4.0,69.28
327,233,Cudrefin,1588,Cudrefin,23.0,69.89
328,234,La Forclaz VD,1866,Ollon,1.0,72.05
329,236,Rougemont,1659,Château-d'Oex,6.0,74.02


#### Distance matrix

In [16]:
distance_matrix_wed = pd.read_csv(repo_url + 'data/dm_mercredi.csv',header=None)
distance_matrix_wed = distance_matrix_wed.iloc[1:,1:]

In [17]:
depot_distances = demand_wed_df.iloc[1:,5:].values.tolist()
depot_distances = [item[0] for item in depot_distances]

# Insert depot as first column
distance_matrix_wed = distance_matrix_wed.copy()
distance_matrix_wed.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_wed.loc[-1] = depot_row
distance_matrix_wed.index = distance_matrix_wed.index + 1
distance_matrix_wed = distance_matrix_wed.sort_index()

# Reset column names and index to integers
distance_matrix_wed.columns = range(distance_matrix_wed.shape[1])
distance_matrix_wed.index = range(distance_matrix_wed.shape[0])

In [18]:
dm_wed = distance_matrix_wed.values.tolist()
for i in range(len(dm_wed)):
    dm_wed[i][i] = 0
dm_wed = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_wed]
dm_wed = [[int(round(float(x))) for x in row] for row in dm_wed]


#### Demand matrix 

In [23]:
demand_wed = demand_wed_df.iloc[1:, 4:5]

In [25]:
d_wed = demand_wed.values.tolist()
d_wed = [int(float(item[0])) for item in d_wed]
d_wed = [0] + d_wed

#### Vehicle Matrix

In [26]:
v_wed_full = v * len(dm_wed)

#### CVRP

In [27]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Mercredi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [31]:

def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_wed, subset_demands=d_wed, capacities=v_wed_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)


main()

    vehicle_id capacity                                              route  \
0            1       91                                       [0, 1306, 0]   
1            4      127                           [0, 1052, 1061, 1041, 0]   
2            7       91                     [0, 1003, 1006, 1081, 1082, 0]   
3         1655      100                                       [0, 1303, 0]   
4         1660      127                           [0, 1585, 1588, 1305, 0]   
..         ...      ...                                                ...   
147       1979      100                                       [0, 1304, 0]   
148       1981       91  [0, 1045, 1409, 1463, 1584, 1587, 1545, 1530, ...   
149       1982       50                                       [0, 1305, 0]   
150       1985      100                                       [0, 1304, 0]   
151      Total                                                               

     distance_km   load  
0              8     74  
1          

### Thursday

In [ ]:
demand_thur_df = pd.read_csv(repo_url + 'demand_thu_full.csv',header=None)

#### Distance Matrix

In [63]:
distance_matrix_thur = pd.read_csv(repo_url + 'data/dm_jeudi.csv',header=None)
distance_matrix_thur = distance_matrix_thur.iloc[1:,1:]

In [65]:

depot_distances = demand_thur_df.iloc[1:, 5:6].values.tolist()
depot_distances = [item[0] for item in depot_distances]

# Insert depot as first column
distance_matrix_thur = distance_matrix_thur.copy()
distance_matrix_thur.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_thur.loc[-1] = depot_row
distance_matrix_thur.index = distance_matrix_thur.index + 1
distance_matrix_thur = distance_matrix_thur.sort_index()

# Reset column names and index to integers
distance_matrix_thur.columns = range(distance_matrix_thur.shape[1])
distance_matrix_thur.index = range(distance_matrix_thur.shape[0])

In [66]:
dm_thur = distance_matrix_thur.values.tolist()
for i in range(len(dm_thur)):
    dm_thur[i][i] = 0
dm_thur = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_thur]
dm_thur = [[int(round(float(x))) for x in row] for row in dm_thur]

#### Demand Matrix

In [67]:
demand_thur = demand_thur_df.iloc[1:, 7:]

In [68]:
d_thur = demand_thur.values.tolist()
d_thur = [int(float(item[0])) for item in d_thur]
d_thur = [0] + d_thur

#### Vehicule Matrix

In [69]:
v_thur_full = v * len(dm_thur)

#### CVRP

In [70]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Jeudi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [71]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_thur, subset_demands=d_thur, capacities=v_thur_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)


main()

   vehicle_id capacity                                              route  \
0           1       91               [0, 1010, 1066, 1081, 1061, 1063, 0]   
1           4      127                                 [0, 1052, 1012, 0]   
2        1493      100                                       [0, 1304, 0]   
3        1498      127  [0, 1404, 1462, 1415, 1464, 1537, 1536, 1526, ...   
4        1499      100                                       [0, 1304, 0]   
..        ...      ...                                                ...   
81       1663       91                                       [0, 1306, 0]   
82       1664       50                                       [0, 1031, 0]   
83       1666      127                                       [0, 1303, 0]   
84       1667      100                                       [0, 1304, 0]   
85      Total                                                               

    distance_km  load  
0            60    89  
1            39   124  
2  

### Friday

In [74]:
demand_fri_df = pd.read_csv(repo_url + 'demand_fri.csv',header=None)

#### Distance Matrix

In [73]:
distance_matrix_fri = pd.read_csv(repo_url + 'data/dm_vendredi.csv',header=None)
distance_matrix_fri = distance_matrix_fri.iloc[1:,1:]

In [76]:
depot_distances = demand_fri_df.iloc[1:, 5:6].values.tolist()
depot_distances = [item[0] for item in depot_distances]

# Insert depot as first column
distance_matrix_fri = distance_matrix_fri.copy()
distance_matrix_fri.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_fri.loc[-1] = depot_row
distance_matrix_fri.index = distance_matrix_fri.index + 1
distance_matrix_fri = distance_matrix_fri.sort_index()

# Reset column names and index to integers
distance_matrix_fri.columns = range(distance_matrix_fri.shape[1])
distance_matrix_fri.index = range(distance_matrix_fri.shape[0])

In [77]:
dm_fri = distance_matrix_fri.values.tolist()
for i in range(len(dm_fri)):
    dm_fri[i][i] = 0
dm_fri = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_fri]
dm_fri = [[int(round(float(x))) for x in row] for row in dm_fri]

#### Demand Matrix

In [78]:
demand_fri = demand_fri_df.iloc[1:, 4:]

In [79]:
d_fri = demand_fri.values.tolist()
d_fri = [int(float(item[0])) for item in d_fri]
d_fri = [0] + d_fri

#### Vehicule Matrix

In [80]:
v_fri_full = v * len(dm_fri)

#### CVRP

In [81]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Vendredi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [82]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_fri, subset_demands=d_fri, capacities=v_fri_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)



main()

   vehicle_id capacity                                              route  \
0           1       91                                 [0, 1003, 1006, 0]   
1           7       91         [0, 1041, 1514, 1526, 1464, 1413, 1374, 0]   
2        1295      100                                       [0, 1304, 0]   
3        1300      127                                 [0, 1117, 1304, 0]   
4        1301      100                                       [0, 1304, 0]   
..        ...      ...                                                ...   
76       1435       91  [0, 1536, 1584, 1586, 1588, 1545, 1404, 1417, ...   
77       1436       50                                       [0, 1036, 0]   
78       1438      127                           [0, 1033, 1070, 1093, 0]   
79       1439      100                                       [0, 1304, 0]   
80      Total                                                               

    distance_km  load  
0            32    90  
1            84    91  
2  

### Saturday

In [84]:
demand_sat_df = pd.read_csv(repo_url + 'demand_sat.csv',header=None)

#### Distance Matrix

In [85]:
distance_matrix_sat = pd.read_csv(repo_url + 'data/dm_samedi.csv',header=None)
distance_matrix_sat = distance_matrix_sat.iloc[1:,1:]

In [86]:
depot_distances = demand_sat_df.iloc[1:, 5:6].values.tolist()
depot_distances = [item[0] for item in depot_distances]

# Insert depot as first column
distance_matrix_sat = distance_matrix_sat.copy()
distance_matrix_sat.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_sat.loc[-1] = depot_row
distance_matrix_sat.index = distance_matrix_sat.index + 1
distance_matrix_sat = distance_matrix_sat.sort_index()

# Reset column names and index to integers
distance_matrix_sat.columns = range(distance_matrix_sat.shape[1])
distance_matrix_sat.index = range(distance_matrix_sat.shape[0])

In [87]:
dm_sat = distance_matrix_sat.values.tolist()
for i in range(len(dm_sat)):
    dm_sat[i][i] = 0
dm_sat = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_sat]
dm_sat = [[int(round(float(x))) for x in row] for row in dm_sat]

#### Demand Matrix

In [88]:
demand_sat = demand_sat_df.iloc[1:, 7:]

In [89]:
d_sat = demand_sat.values.tolist()
d_sat = [int(float(item[0])) for item in d_sat]
d_sat = [0] + d_sat

#### Vehicule Matrix

In [92]:
v_sat_full = v * len(dm_sat)

#### CVRP

In [93]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Samedi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [94]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_sat, subset_demands=d_sat, capacities=v_sat_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)

main()

   vehicle_id capacity                                 route  distance_km  \
0        1342      127                    [0, 1054, 1033, 0]           22   
1        1343      100                          [0, 1304, 0]           10   
2        1348      127                    [0, 1035, 1033, 0]           18   
3        1349      100                          [0, 1305, 0]            6   
4        1354      127  [0, 1304, 1313, 1354, 1355, 1439, 0]           59   
..        ...      ...                                   ...          ...   
72       1487      100                          [0, 1303, 0]            2   
73       1489       91                          [0, 1305, 0]            6   
74       1492      127                          [0, 1303, 0]            2   
75       1493      100                          [0, 1304, 0]           10   
76      Total                                                        2333   

    load  
0    125  
1     96  
2    124  
3     97  
4    120  
..   ... 

## Scenario 2 - Full demand

### Wednesday

#### Demand matrix

In [96]:
demand_wed = demand_wed_df.iloc[1:,4:5]

In [97]:
d_wed = demand_wed.values.tolist()
d_wed = [int(float(item[0])) for item in d_wed]
d_wed = [0] + d_wed

#### CVRP

In [119]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_wed, subset_demands=d_wed1, capacities=v_wed_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)


main()

    vehicle_id capacity                           route  distance_km   load
0         1702      127              [0, 1377, 1033, 0]           26    127
1         1703      100                    [0, 1304, 0]           10     96
2         1708      127  [0, 1090, 1073, 1068, 1033, 0]           48    124
3         1709      100                    [0, 1304, 0]           10     96
4         1714      127              [0, 1003, 1033, 0]           32    122
..         ...      ...                             ...          ...    ...
121       1981       91              [0, 1035, 1034, 0]           11     85
122       1982       50                    [0, 1305, 0]            6     36
123       1984      127                    [0, 1303, 0]            2     98
124       1985      100                    [0, 1304, 0]           10     96
125      Total                                                  3113  11838

[126 rows x 5 columns]


In [120]:
pd.read_csv(repo_url + 'Tickets_du_Mercredi.csv', header=None)

,0,1,2,3,4
0,index,Commune,NPA,Commune d'annonce / District,wed_tickets
1,0,Penthaz,1303,Penthaz,100.0
2,0,Penthaz,1303,Penthaz,100.0
3,0,Penthaz,1303,Penthaz,46.0
4,1,Penthalaz,1305,Penthalaz,100.0
...,...,...,...,...,...
326,232,Chesières,1885,Ollon,4.0
327,233,Cudrefin,1588,Cudrefin,23.0
328,234,La Forclaz VD,1866,Ollon,1.0
329,236,Rougemont,1659,Château-d'Oex,6.0


In [122]:
pd.read_csv(repo_url + 'data/dm_mercredi.csv')

,Unnamed: 0,0,0.1,0.2,1,1.1,1.2,2,3,4,...,226,227,228,230,231,232,233,234,236,237
0,0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,64.7759,66.2471,67.3176,68.5917,68.6752,69.3386,69.9441,72.1044,74.0778,77.0503
1,0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,64.7759,66.2471,67.3176,68.5917,68.6752,69.3386,69.9441,72.1044,74.0778,77.0503
2,0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,64.7759,66.2471,67.3176,68.5917,68.6752,69.3386,69.9441,72.1044,74.0778,77.0503
3,1,2.6786,2.6786,2.6786,inf,inf,inf,3.9804,3.0074,5.6066,...,66.9319,64.3614,69.4735,70.7477,70.8312,71.4946,68.0584,74.2604,76.2338,79.2062
4,1,2.6786,2.6786,2.6786,inf,inf,inf,3.9804,3.0074,5.6066,...,66.9319,64.3614,69.4735,70.7477,70.8312,71.4946,68.0584,74.2604,76.2338,79.2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,232,69.3386,69.3386,69.3386,71.4946,71.4946,71.4946,69.4933,71.6967,69.3463,...,13.2488,107.0621,4.7761,25.1109,48.8300,inf,112.0229,10.1314,51.9416,9.3360
326,233,69.9441,69.9441,69.9441,68.0584,68.0584,68.0584,69.8029,65.9490,69.6334,...,107.9229,5.5109,110.0019,112.2001,80.1927,112.0229,inf,115.5113,85.9790,120.4571
327,234,72.1044,72.1044,72.1044,74.2604,74.2604,74.2604,72.2591,74.4625,72.1121,...,16.2099,109.8279,7.4100,24.8423,28.5708,10.1314,115.5113,inf,31.4998,18.1614
328,236,74.0778,74.0778,74.0778,76.2338,76.2338,76.2338,74.2325,76.4359,74.0855,...,46.2638,80.2956,38.5940,55.4904,8.6894,51.9416,85.9790,31.4998,inf,57.1422


In [123]:
demand_wed_df

,0,1,2,3,4,5,6,7
0,index,Commune,NPA,Commune d'annonce / District,full_demand_wed,distance_to_venoge_km,P_Y,adjusted_demand_wed
1,0,Penthaz,1303,Penthaz,100.0,0.74,0.9792044207837601,98
2,0,Penthaz,1303,Penthaz,100.0,0.74,0.9792044207837601,98
3,0,Penthaz,1303,Penthaz,46.0,0.74,0.9792044207837601,45
4,1,Penthalaz,1305,Penthalaz,100.0,2.72,0.969409570578176,97
...,...,...,...,...,...,...,...,...
326,232,Chesières,1885,Ollon,4.0,69.28,5.242890847627327e-05,0
327,233,Cudrefin,1588,Cudrefin,23.0,69.89,4.6407642245884355e-05,0
328,234,La Forclaz VD,1866,Ollon,1.0,72.05,3.012876697309509e-05,0
329,236,Rougemont,1659,Château-d'Oex,6.0,74.02,2.0317655560662326e-05,0
